In [1]:
import mlflow

mlflow.set_tracking_uri(uri="http://<host>:<port>")

ModuleNotFoundError: No module named 'mlflow'

In [2]:
!pip install mlflow

   ---------------------------------------- 0.0/20.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/20.2 MB 1.3 MB/s eta 0:00:16
   ---------------------------------------- 0.1/20.2 MB 812.7 kB/s eta 0:00:25
   ---------------------------------------- 0.1/20.2 MB 751.6 kB/s eta 0:00:27
   ---------------------------------------- 0.1/20.2 MB 774.0 kB/s eta 0:00:26
   ---------------------------------------- 0.2/20.2 MB 807.1 kB/s eta 0:00:25
   ---------------------------------------- 0.2/20.2 MB 787.7 kB/s eta 0:00:26
   ---------------------------------------- 0.2/20.2 MB 724.0 kB/s eta 0:00:28
    --------------------------------------- 0.3/20.2 MB 714.4 kB/s eta 0:00:28
    --------------------------------------- 0.3/20.2 MB 759.5 kB/s eta 0:00:27
    --------------------------------------- 0.3/20.2 MB 776.5 kB/s eta 0:00:26
    --------------------------------------- 0.4/20.2 MB 782.8 kB/s eta 0:00:26
    --------------------------------------- 0.4/20.2 MB 748.8 k

In [3]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)


In [4]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )


2024/05/05 21:41:44 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Quickstart' does not exist. Creating a new experiment.
C:\Users\rohith.thokala\AppData\Local\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'tracking-quickstart'.
2024/05/05 21:41:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 1
Created version '1' of model 'tracking-quickstart'.


In [5]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]


2024/05/05 21:42:17 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
